In [2]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
from PIL import Image
from skimage.transform import resize
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, MobileNetV2, InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import h5py
import time
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import seaborn as sns

# Define image parameters
img_width, img_height = 64, 64

# Load and preprocess image data
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = Image.open(img_path)

            # Convert image to RGB mode
            img = img.convert("RGB")

            img = img.resize((img_width, img_height))
            img_array = np.array(img)

            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

base_folder = os.path.join('../..', 'Augmented_Dataset')

# Load images from "train" folder
test_folder = os.path.join(base_folder, 'test')
test_images, test_labels = load_images_from_folder(test_folder)

# Convert string labels to integer class indices
label_encoder = LabelEncoder()
test_labels_encoded = label_encoder.fit_transform(test_labels)

# Define the path to the trained_model_parameters folder
parameter_folder = os.path.join('..', '..', 'trained_model_parameters', 'MobileNetV2')

model = tf.keras.models.load_model(os.path.join(parameter_folder, 'MobileNetV2_best_model.h5'))
model._name = "MobileNetV2"

import time
# Make predictions on the validation set
start_time = time.time()
test_predictions = model.predict(test_images)
end_time = time.time()
test_pred_labels = np.argmax(test_predictions, axis=1)
# Calculate the inference time
inference_time = end_time - start_time

# Print the Confusion Matrix using confusion_matrix element in Sklearn.
from sklearn import metrics
print("Confusion Matrix: ")
print(metrics.confusion_matrix(test_labels_encoded, test_pred_labels))
print("\n")

# Print the Classification Report using classification_report element in Sklearn.
from sklearn.metrics import classification_report
print("Classification Report: ")
print( classification_report(test_labels_encoded, test_pred_labels))

# Confusion Matrix
conf_matrix = confusion_matrix(test_labels_encoded, test_pred_labels)

# Calculate and print the accuracy, precision, recall, and F1-score
accuracy = accuracy_score(test_labels_encoded, test_pred_labels)
precision = precision_score(test_labels_encoded, test_pred_labels, average='macro')
recall = recall_score(test_labels_encoded, test_pred_labels, average='macro')
f1 = f1_score(test_labels_encoded, test_pred_labels, average='macro')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")

print("\n")

print("Average inference time per sample (second):", inference_time / len(test_images))

print("\n")

# Print out the layer structure
model.summary()

from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph

def get_flops(model):
    concrete = tf.function(lambda inputs: model(inputs))
    concrete_func = concrete.get_concrete_function(
        [tf.TensorSpec([1, *inputs.shape[1:]]) for inputs in model.inputs])
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete_func)
    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        return flops.total_float_ops

print("\n")    
print("FLOPs: ", get_flops(model))




102/102 [==============================] - 3s 19ms/step
Confusion Matrix: 
[[313   7   3   0  22   0   5   2   1   0   3]
 [  1 299   1   1   4   1   1   1   1   0   0]
 [  3   9 363   2   7   0   1   1   1   1   3]
 [  1   2   0 339   5   1   1   0   1   0   0]
 [  1   5   2   3 348   0   0   3   0   0   1]
 [  0   0   0   0   0 214   1   0   1   2   1]
 [  0   0   0   0   3  12 200   0   3  11   0]
 [  2   0   0   0   0   1   1 250   0   0   0]
 [  0   0   0   1   0   0   0   3 267   0   3]
 [  0   0   0   0   0   0   0   3   3 380   0]
 [  1   0   4   3   5   0   0   1   0   1 111]]


Classification Report: 
              precision    recall  f1-score   support

           0       0.97      0.88      0.92       356
           1       0.93      0.96      0.95       310
           2       0.97      0.93      0.95       391
           3       0.97      0.97      0.97       350
           4       0.88      0.96      0.92       363
           5       0.93      0.98      0.96       219
